# Testing Model Inheritance

**Authorship:**
Adam Klie, *04/06/2022*
***
**Description:**
Notebook to test the inheritance of models.
***

In [1]:
# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu

Global seed set to 13
Global seed set to 13
Global seed set to 13


In [2]:
eu.__version__

'0.0.0'

# Instantiate models

In [3]:
eu.models.DeepSEA?

Init signature:
eu.models.DeepSEA(
    input_len=1000,
    output_dim=1,
    strand='ss',
    task='regression',
    aggr=None,
    conv_kwargs={},
    fc_kwargs={},
)
Docstring:      Hooks to be used in LightningModule.
Init docstring:
Generates a PyTorch module with architecture matching the convnet part of DeepSea. Default parameters are those specified in the DeepSea paper
Parameters
----------
input_len : int, input sequence length
channels : list-like or int, channel width for each conv layer. If int each of the three layers will be the same channel width
conv_kernels : list-like or int, conv kernel size for each conv layer. If int will be the same for all conv layers
pool_kernels : list-like or int, maxpooling kernel size for the first two conv layers. If int will be the same for all conv layers
dropout_rates : list-like or float, dropout rates for each conv layer. If int will be the same for all conv layers
File:           /mnt/beegfs/users/aklie/projects/EUGENE/eugene/models/_

In [5]:
#model = eu.models.FCN(input_len=66, output_dim=1)
#model = eu.models.CNN(input_len=66, conv_kwargs={"channels": [4, 32], "conv_kernels": [4], "pool_kernels": [2]}, output_dim=1) 
#model = eu.models.RNN(input_len=66, rnn_kwargs={"output_dim":1}, output_dim=1)
#model = eu.models.Hybrid(input_len=66, conv_kwargs={"channels": [4, 32], "conv_kernels": [4], "pool_kernels": [2]}, rnn_kwargs={"output_dim":1}, output_dim=1)
#model = eu.models.DeepBind(input_len=66, output_dim=1)
model = eu.models.DeepSEA()
model

DeepSEA(
  (r_squared): R2Score()
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 320, kernel_size=(8,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(320, 480, kernel_size=(8,), stride=(1,))
      (5): ReLU()
      (6): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
      (8): Conv1d(480, 960, kernel_size=(8,), stride=(1,))
      (9): ReLU()
      (10): Dropout(p=0.5, inplace=False)
    )
  )
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=50880, out_features=925, bias=True)
      (1): ReLU()
      (2): Linear(in_features=925, out_features=1, bias=True)
    )
  )
)

In [6]:
print(type(model))
print(isinstance(model, eu.models.base.BaseModel))

<class 'eugene.models._sota_models.DeepSEA'>
True


# Load data

In [7]:
sdata = eu.datasets.random1000()
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
seqs_annot: 'TARGETS'
pos_annot: PyRanges object with 1456 features

In [8]:
eu.pp.train_test_split_data(sdata, kwargs = {"split": 0.8})

SeqData object modified:
    seqs_annot:
        + TRAIN


# Train (and time) with PyTorch Lightning

In [13]:
eu.train.fit(model, sdata=sdata, epochs=3, num_workers=4, log_dir="../_logs", out_dir="../_out/")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type                      | Params
--------------------------------------------------------
0 | r_squared | R2Score                   | 0     
1 | max_pool  | MaxPool1d                 | 0     
2 | avg_pool  | AvgPool1d                 | 0     
3 | convnet   | BasicConv1D               | 272   
4 | fcn       | BasicFullyConnectedModule | 146 K 
--------------------------------------------------------
147 K     Trainable params
0         Non-trainable params
147 K     Total params
0.588     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 13
/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

SeqData object modified:
    seqs_annot:
        + PREDICTIONS


---